此项目参照`红白黑'的基线代码运行（个人主页[https://aistudio.baidu.com/aistudio/personalcenter/thirdview/277680](http://)）

参考项目链接：[https://aistudio.baidu.com/aistudio/projectdetail/1938000?channelType=0&channel=0](http://)

checkpoint保存在PaddleSeg/output/best_model/model.pdparams

# 一、项目准备
将数据集划分好并放在PaddleSeg/data目录下



In [2]:
# 解压PaddleSeg压缩包
!unzip -oq data/data88946/PaddleSeg.zip -d /home/aistudio/
# 修改文件名
!mv PaddleSeg-release-v2.0 PaddleSeg

mv: cannot move 'PaddleSeg-release-v2.0' to 'PaddleSeg/PaddleSeg-release-v2.0': Directory not empty


In [3]:
# 解压数据集到PaddleSeg目录下的data文件夹
!unzip -oq /home/aistudio/data/data86770/seg.zip -d PaddleSeg/data


# 二、数据预处理


划分数据集，验证集比例为0.3



In [ ]:
# 保证路径为初始路径
%cd /home/aistudio

# 划分数据
!python utils/dataset_splited.py

# 转换标签--预处置
!python utils/dataset_pretrans.py

/home/aistudio
————开始数据清洗划分————
The Split Params: train_percent=0.70
Image Dir Has Ready!
Label Dir Has Ready!
Processing Test Split: 100%|█████████████████| 400/400 [00:03<00:00, 111.13it/s]
完成数据集划分：
- 数据根目录: PaddleSeg/data
	 - 子目录: Image: 存放真实图片
		 - to_train: 训练用 , lens:560
		 - to_val: 验证用 , lens:240
		 - to_test: 测试用(提交预测) , lens:400
	 - 子目录: Label: 存放标注图片
		 - to_train: 训练用 , lens:560
		 - to_val: 验证用 , lens:240
		 - to_test: 测试用(保存提交预测) , lens:0
————开始数据预处理转换————
转换说明:
	 1. 默认标签为255与0，为了训练方便，将255转换为1，变成2分类问题
	 2. 新标签0与1，预测结束进行后处理即可得到赛题需要的结果
100%|█████████████████████████████████████████| 800/800 [00:51<00:00, 15.62it/s]


# 三、下载依赖项

> 平台可以不用下载，但是如果在本地可能需要执行这一步

In [7]:
# 下载依赖项，保证PaddleSeg正常运行
%cd PaddleSeg
%pwd
!pip install -r requirements.txt

/home/aistudio/PaddleSeg
Looking in indexes: https://mirror.baidu.com/pypi/simple/


# 四、开始构建比赛模型

## 1.导入需要的库

In [8]:
# 当前套件下切换目录到PaddleSeg下，才能使用paddleseg
%cd PaddleSeg
%pwd

import paddle                                     # paddle基本框架
from paddleseg import models as M                 # paddleseg的模型库--对应模型源代码：PaddleSeg/paddleseg/models
from paddleseg.models import backbones as B       # 分割模型需要的骨干网络--对应模型源代码：PaddleSeg/paddleseg/models/backbones
from paddleseg.models import losses as L          # 分割模型需要的损失函数--对应模型源代码：PaddleSeg/paddleseg/models/losses
from paddleseg import transforms as T             # 分割模型需要的数据预处理方法(图像)--对应模型源代码：PaddleSeg/paddleseg/transforms/transforms.py
from paddleseg.datasets import OpticDiscSeg       # paddleseg对应的数据加载机制--对应模型源代码：PaddleSeg/paddleseg/datasets/optic_disc_seg.py
from paddleseg.core import train, evaluate, predict  # 训练、评估、预测接口--对应模型源代码：PaddleSeg/paddleseg/core

import os                                         # 必要的文件处理

[Errno 2] No such file or directory: 'PaddleSeg'
/home/aistudio/PaddleSeg


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


## 2.创建模型与Dataset

In [9]:
# # 实例创建一个模型——当前演示基线模型EMANet
# # 其它模型可前往PaddleSeg/paddleseg/models下查看相应的其它.py文件查阅参数，进行配置
# # 也可通过help(M.EMANet)查看类文档描述
# # 必要的，可以参考PaddleSeg/configs中同名模型yml文件配置
# model = M.EMANet(
#                  num_classes=2,                  # 类别数，这里已经转换为2分类问题了
#                  backbone=B.ResNet50_vd(),       # 选用骨干网络--注意骨干网络和backbone_indices的搭配(注意传入的是实例化的对象哦，不要传成类了)
#                  backbone_indices=[2, 3]         # backbone流向分割部分的特征通道index，必要时可以使用[1,], [1, 2], [2, 3], [1, 3], [1, 2, 3]
# )

#此处为了项目便于修改，把模型参数放在训练命令处

2021-07-14 09:04:51 [INFO]	No pretrained model to load, ResNet_vd will be trained from scratch.


In [10]:
# 配置dataset以及相应的transform
train_transforms = [
    T.RandomHorizontalFlip(),          # 水平翻转
    T.Resize(target_size=(800, 800)),
    T.Normalize()
]

# 验证处理可以不添加额外的方式，确定形状即可
eval_transforms = [
    T.Resize(target_size=(800, 800)),  # 缩放大小
    T.Normalize()
]

# 创建数据集
train_dataset = OpticDiscSeg(
    dataset_root='data',             # PaddleSeg/data目录下存在与mode对应的train_list.txt
    transforms=train_transforms,
    mode='train'                     # mode不要写错了
)

eval_dataset = OpticDiscSeg(
    dataset_root='data',
    transforms=eval_transforms,
    mode='val'
)

## 3.配置学习率与损失计算方式

In [11]:
base_lr = 0.01          
# 多项式学习率，由decay_steps决定base_lr到base_lr*0.01的间隔迭代次数
lr = paddle.optimizer.lr.PolynomialDecay(base_lr, power=0.9, decay_steps=2000, end_lr=base_lr*0.01)

# 优化器--可以换用Adam
# 正则项可以调高一些
optimizer = paddle.optimizer.Momentum(lr, parameters=model.parameters(), momentum=0.9, weight_decay=4.0e-5)
#momentum，yyds

In [12]:
# paddleseg支持多损失，所以采用字典的方式配置损失
losses = {}  # 创建loss字典
# losses['types'] = [L.CrossEntropyLoss(), L.DiceLoss()]  # 将需要的多个损失进行添加
# losses['types'] = [L.CrossEntropyLoss()]
losses['types'] = [ L.DiceLoss()] 
# 每一个损失计算的结果的权重
# all_loss = losses['types'][0] * losses['coef'][0] + losses['types'][2] * losses['coef'][2]
losses['coef'] = [ 2.0] 
#此处仅训练单一loss

## 4.开始训练

In [13]:
# 实例创建一个模型——当前演示基线模型EMANet
# 其它模型可前往PaddleSeg/paddleseg/models下查看相应的其它.py文件查阅参数，进行配置
# 也可通过help(M.EMANet)查看类文档描述
# 必要的，可以参考PaddleSeg/configs中同名模型yml文件配置
model = M.UNet(
                 num_classes=2,                  # 类别数，这里已经转换为2分类问题了
                #  backbone=B.ResNet50_vd(),       # 选用骨干网络--注意骨干网络和backbone_indices的搭配(注意传入的是实例化的对象哦，不要传成类了)
                #  backbone_indices=[2, 3]         # backbone流向分割部分的特征通道index，必要时可以使用[1,], [1, 2], [2, 3], [1, 3], [1, 2, 3]
)
#接上文，将基线模型EMANet改为Unet模型

In [19]:
#开始训练
train(
    model=model,                        # 创建的模型
    train_dataset=train_dataset,        # 训练数据集
    val_dataset=eval_dataset,           # 验证数据集
    optimizer=optimizer,                # 优化器
    save_dir='output',                  # 保存路径--不必该，否则后边程序也需要修正
    iters=24000,                         # 训练迭代次数（这里不是轮次）
    batch_size=1,                       # 批大小
    save_interval=2000,                  # 验证+保存的迭代周期
    log_iters=10,                       # 日志输出的迭代周期
    num_workers=0,                      # 多线程关闭(0)--在平台上开启可能会断掉
    losses=losses,                      # 损失字典
    use_vdl=True)                       # 是否记录训练参数

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:238: UserWarning: The dtype of left and right variables are not the same, left dtype is VarType.FP32, but right dtype is VarType.BOOL, the right dtype will convert to VarType.FP32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


2021-06-21 21:32:16 [INFO]	[TRAIN] epoch=1, iter=10/24000, loss=0.8801, lr=0.009960, batch_cost=0.2620, reader_cost=0.09308, ips=3.8163 samples/sec | ETA 01:44:46
2021-06-21 21:32:19 [INFO]	[TRAIN] epoch=1, iter=20/24000, loss=0.5061, lr=0.009915, batch_cost=0.2874, reader_cost=0.12349, ips=3.4800 samples/sec | ETA 01:54:50
2021-06-21 21:32:22 [INFO]	[TRAIN] epoch=1, iter=30/24000, loss=0.5538, lr=0.009871, batch_cost=0.2691, reader_cost=0.10634, ips=3.7163 samples/sec | ETA 01:47:30
2021-06-21 21:32:25 [INFO]	[TRAIN] epoch=1, iter=40/24000, loss=0.3451, lr=0.009826, batch_cost=0.2695, reader_cost=0.10394, ips=3.7105 samples/sec | ETA 01:47:37
2021-06-21 21:32:27 [INFO]	[TRAIN] epoch=1, iter=50/24000, loss=0.5517, lr=0.009781, batch_cost=0.2740, reader_cost=0.10916, ips=3.6491 samples/sec | ETA 01:49:23
2021-06-21 21:32:30 [INFO]	[TRAIN] epoch=1, iter=60/24000, loss=0.5397, lr=0.009737, batch_cost=0.2572, reader_cost=0.09432, ips=3.8875 samples/sec | ETA 01:42:38
2021-06-21 21:32:33 [I

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:238: UserWarning: The dtype of left and right variables are not the same, left dtype is VarType.INT32, but right dtype is VarType.BOOL, the right dtype will convert to VarType.INT32
  format(lhs_dtype, rhs_dtype, lhs_dtype))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:238: UserWarning: The dtype of left and right variables are not the same, left dtype is VarType.INT64, but right dtype is VarType.BOOL, the right dtype will convert to VarType.INT64
  format(lhs_dtype, rhs_dtype, lhs_dtype))
240/240 [==============================] - 36s 150ms/step - batch_cost: 0.1487 - reader cost: 0.13


2021-06-21 21:41:49 [INFO]	[EVAL] #Images=240 mIoU=0.7000 Acc=0.9878 Kappa=0.5778 
2021-06-21 21:41:49 [INFO]	[EVAL] Class IoU: 
[0.9877 0.4123]
2021-06-21 21:41:49 [INFO]	[EVAL] Class Acc: 
[0.9913 0.7033]
2021-06-21 21:41:51 [INFO]	[EVAL] The model with the best validation mIoU (0.7000) was saved at iter 2000.
2021-06-21 21:41:52 [INFO]	[TRAIN] epoch=4, iter=2010/24000, loss=0.1479, lr=0.000100, batch_cost=0.1730, reader_cost=0.00026, ips=5.7820 samples/sec | ETA 01:03:23
2021-06-21 21:41:55 [INFO]	[TRAIN] epoch=4, iter=2020/24000, loss=0.0975, lr=0.000100, batch_cost=0.2763, reader_cost=0.11278, ips=3.6192 samples/sec | ETA 01:41:13
2021-06-21 21:41:58 [INFO]	[TRAIN] epoch=4, iter=2030/24000, loss=0.0741, lr=0.000100, batch_cost=0.2524, reader_cost=0.08722, ips=3.9619 samples/sec | ETA 01:32:25
2021-06-21 21:42:00 [INFO]	[TRAIN] epoch=4, iter=2040/24000, loss=0.2567, lr=0.000100, batch_cost=0.2665, reader_cost=0.09317, ips=3.7519 samples/sec | ETA 01:37:33
2021-06-21 21:42:03 [INFO]

240/240 [==============================] - 36s 151ms/step - batch_cost: 0.1493 - reader cost: 0.134


...

240/240 [==============================] - 38s 156ms/step - batch_cost: 0.1551 - reader cost: 0.139


2021-06-21 23:19:23 [INFO]	[EVAL] #Images=240 mIoU=0.7232 Acc=0.9875 Kappa=0.6229 
2021-06-21 23:19:23 [INFO]	[EVAL] Class IoU: 
[0.9874 0.459 ]
2021-06-21 23:19:23 [INFO]	[EVAL] Class Acc: 
[0.9934 0.64  ]
2021-06-21 23:19:24 [INFO]	[EVAL] The model with the best validation mIoU (0.7321) was saved at iter 20000.
2021-06-21 23:19:26 [INFO]	[TRAIN] epoch=40, iter=22010/24000, loss=0.0444, lr=0.000100, batch_cost=0.1748, reader_cost=0.00839, ips=5.7221 samples/sec | ETA 00:05:47
2021-06-21 23:19:29 [INFO]	[TRAIN] epoch=40, iter=22020/24000, loss=0.0556, lr=0.000100, batch_cost=0.2874, reader_cost=0.12218, ips=3.4789 samples/sec | ETA 00:09:29
2021-06-21 23:19:32 [INFO]	[TRAIN] epoch=40, iter=22030/24000, loss=0.0749, lr=0.000100, batch_cost=0.2746, reader_cost=0.10992, ips=3.6422 samples/sec | ETA 00:09:00
2021-06-21 23:19:34 [INFO]	[TRAIN] epoch=40, iter=22040/24000, loss=0.0826, lr=0.000100, batch_cost=0.2571, reader_cost=0.09126, ips=3.8890 samples/sec | ETA 00:08:23
2021-06-21 23:19:

240/240 [==============================] - 38s 159ms/step - batch_cost: 0.1569 - reader cost: 0.141


2021-06-21 23:29:19 [INFO]	[EVAL] #Images=240 mIoU=0.6962 Acc=0.9881 Kappa=0.5701 
2021-06-21 23:29:19 [INFO]	[EVAL] Class IoU: 
[0.988  0.4043]
2021-06-21 23:29:19 [INFO]	[EVAL] Class Acc: 
[0.9909 0.7408]
2021-06-21 23:29:20 [INFO]	[EVAL] The model with the best validation mIoU (0.7321) was saved at iter 20000.
<class 'paddle.nn.layer.conv.Conv2D'>'s flops has been counted
Customize Function has been applied to <class 'paddle.nn.layer.norm.SyncBatchNorm'>
Cannot find suitable count function for <class 'paddle.nn.layer.pooling.MaxPool2D'>. Treat it as zero FLOPs.
Total Flops: 303476473856     Total Params: 13404354


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:143: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:238: UserWarning: The dtype of left and right variables are not the same, left dtype is VarType.FP32, but right dtype is VarType.INT32, the right dtype will convert to VarType.FP32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


## 5.开始预测

预测的配置略微不同，需要读取`test_list.txt`中的文件进入list中，然后传入list以及Image_dir进行预测

> 前面的训练与验证是通过给dir，自动搜寻，这里不一样，要注意一下哦

In [14]:
%cd PaddleSeg
test_list = []
test_root = 'data'      # 之前划分数据图像保存的根路径
with open('data/test_list.txt') as f: 
    for i in f.readlines():
        test_list.append(os.path.join(test_root, i[:-1]))   # 逐行写入，-1是为了去掉 \n

# 预测的transform也略有不同，前边时list，这里需要严格的传入transform格式
# 利用Compose实现多处理
transforms = T.Compose([
    T.Resize(target_size=(800, 800)),
    T.Normalize()
])

[Errno 2] No such file or directory: 'PaddleSeg'
/home/aistudio/PaddleSeg


使用`predict`接口进行预测

In [15]:
predict(
        model,                                           # 创建的模型
        model_path='output/best_model/model.pdparams',   # 模型参数
        transforms=transforms,                           # 数据处理方式--尽量避免高斯那些操作，在这里效果不好
        image_list=test_list,                            # 上边生成的图片list
        image_dir=test_root,                             # 图片保存的形式--保证预测结果保存在同名的结构中，但不是在test_root目录下，而是output/results中
        save_dir='output/results'                        # 保存路径——PaddleSeg/output/results/pseudo_color_prediction：为真实预测结果
    )

2021-07-14 09:05:58 [INFO]	Loading pretrained model from output/best_model/model.pdparams
2021-07-14 09:05:59 [INFO]	There are 112/112 variables loaded into UNet.
2021-07-14 09:05:59 [INFO]	Start to predict...


400/400 [==============================] - 126s 315ms/ste


## 6.后处理并生成提交文件

In [27]:
%cd /home/aistudio
!python utils/post_process.py

/home/aistudio
————开始提交结果前的后处理————
100%|█████████████████████████████████████████| 400/400 [00:22<00:00, 18.16it/s]
后处理完成(cost: 22.039443969726562 s)！


In [28]:
# 复制文件到最顶层目录
!cp -r PaddleSeg/output/results/pseudo_color_prediction/Image/ Disc_Segmentation
# 压缩文件
!zip -r Disc_Segmentation.zip Disc_Segmentation
# 删除复制的文件
#!rm -rf Disc_Segmentation

  adding: Disc_Segmentation/ (stored 0%)
  adding: Disc_Segmentation/T0352.png (deflated 75%)
  adding: Disc_Segmentation/T0260.png (deflated 98%)
  adding: Disc_Segmentation/T0177.png (deflated 95%)
  adding: Disc_Segmentation/T0354.png (deflated 68%)
  adding: Disc_Segmentation/T0204.png (deflated 73%)
  adding: Disc_Segmentation/T0193.png (deflated 68%)
  adding: Disc_Segmentation/T0047.png (deflated 75%)
  adding: Disc_Segmentation/T0052.png (deflated 75%)
  adding: Disc_Segmentation/T0367.png (deflated 76%)
  adding: Disc_Segmentation/T0317.png (deflated 74%)
  adding: Disc_Segmentation/T0329.png (deflated 98%)
  adding: Disc_Segmentation/T0368.png (deflated 98%)
  adding: Disc_Segmentation/T0186.png (deflated 75%)
  adding: Disc_Segmentation/T0030.png (deflated 83%)
  adding: Disc_Segmentation/T0141.png (deflated 37%)
  adding: Disc_Segmentation/T0208.png (deflated 98%)
  adding: Disc_Segmentation/T0213.png (deflated 98%)
  adding: Disc_Segmentation/T0150.png (deflated 77%)
  add

In [23]:
!ls

2105739.ipynb  data  PaddleSeg	utils  work


此项目参照红白黑的基线代码运行（个人主页[https://aistudio.baidu.com/aistudio/personalcenter/thirdview/277680](http://)）

参考项目链接：[https://aistudio.baidu.com/aistudio/projectdetail/1938000?channelType=0&channel=0](http://)
